In [2]:
# QUERY RESOURCE AVAILIBILITY 
# Get Avertisement from Proxy

def clear(self):
    viewer.clear_output()
    viewer.clear_output()
    with viewer:
        viewer.append_stdout('PROJECT VIEWER')
        display(widgets.HBox([ad_res_button, rec_top_button, build_req_button, clear_button]))
    
clear_button = widgets.Button(
    description='Clear Viewer',
    disabled=False,
    tooltip='Click to clear viewer'
)
clear_button.on_click(clear)


def advertisement(self):
    status, advertised_topology = slice_manager.resources()
    with viewer:
        print(f"Status: {status}")
        print(f"Toplogy: {advertised_topology}")
        advertised_topology.draw
    
ad_res_button = widgets.Button(
    description='Advertised Resources',
    disabled=False,
    tooltip='Click to view site availible resources'
)
ad_res_button.on_click(advertisement)

# Display Resource Topology

def resource_topology(self):
    status, advertised_topology = slice_manager.resources()
    with viewer:
        display(advertised_topology.draw())
        
rec_top_button = widgets.Button(
    description='Display Topology',
    disabled=False,
    tooltip='Click to view advertised resource topology',
)
rec_top_button.on_click(resource_topology)

# Build Slice Request

# Create Node builder

node_name = widgets.Text(
    placeholder='enter name',
    description='Node Name:',
    disabled=False
)

site_select = widgets.Select(
    options=['RENC', 'UKY', 'LBNL', 'STAR'],
    value='UKY',
    description='Site:',
    disabled=False
)

core_slider = widgets.IntSlider(min=2, max=32, step=2, value=4, description='Cores')
ram_slider = widgets.IntSlider(min=2, max=64, step=2, value=16, description='RAM')
disk_slider = widgets.FloatLogSlider(base=2, min=5, max=17, step=1, value=128,description='Disk')

img_ref_select = widgets.Select(
    options=['qcow2', 'qchicken2', 'qpig2'],
    value='qcow2',
    description='Image Type:',
    disabled=False
)

img_type_select = widgets.Select(
    options=['default_ubuntu_20', 'default_arch_20', 'default_kali_21'],
    value='default_ubuntu_20',
    description='Image Reference:',
    disabled=False
)
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, LinkType, Layer

experiment = ExperimentTopology()
slice_name="widget_test_slice"

def submit_request(self):
    slice_graph = experiment.serialize()

    # Request slice from Orchestrator
    status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)

    print("Request {}".format(status))
    slice_id=reservations[0].slice_id

    print("Reservations: {}".format(reservations))
    print("Slice ID: {}".format(slice_id))

    viewer.append_stdout("Request {}".format(status))
    slice_id=reservations[0].slice_id

    viewer.append_stdout("Reservations: {}".format(reservations))
    viewer.append_stdout("Slice ID: {}".format(slice_id))
    viewer.display()
    
submit_request_button = widgets.Button(
    decsription='Submit Request',
    disabled=False,
    tooltip='Click to submit request'
)
submit_request_button.on_click(submit_request)

def preview(self):
    n1 = experiment.add_node(name=node_name.value, site=site_select.value)
    cap = Capacities().set_fields(core=core_slider.value, ram=ram_slider.value, disk=disk_slider.value)
    n1.set_properties(capacities=cap)
    with viewer:
        print("Preview")
        viewer.append_stdout('Node Parameters: {} - {}\n'.format(node_name.value, site_select.value))
        viewer.append_stdout('Cores: {}\n RAM: {}\n Disk Size(G): {}\n'.format(core_slider.value, ram_slider.value, disk_slider.value))
        viewer.append_stdout('Image type: {}\n Image Ref: {}\n'.format(img_ref_select.value, img_type_select.value))
        experiment.draw()
        display(submit_request_button)

preview_request = widgets.Button(
    description='Preview Request',
    disabled=False,
    tooltip='click to build node',
)
preview_request.on_click(preview)


nvme_name = widgets.Text(
    placeholder='enter name',
    description='NVME Name:',
    disabled=False
)

nvme_select = widgets.Select(
    options=['P4510'],
    value='P4510',
    description='NVME Type:',
    disabled=False
)

gpu_name = widgets.Text(
    placeholder='enter name',
    description='GPU Name:',
    disabled=False
)

gpu_select = widgets.Select(
    options=['RTX6000', 'Tesla T4'],
    value='Tesla T4',
    description='GPU type',
    disabled=False
)

nic_name = widgets.Text(
    placeholder='enter name',
    description='NIC Name:',
    disabled=False
)

nic_select = widgets.Select(
    options=['ConnectX-5', 'ConnectX-6'],
    value='ConnectX-5',
    description='NIC Type:',
    disabled=False
)


def submit_component(self):
    try:
        node1_nvme = node1.add_component(ctype=ComponentType.NVME, model=nvme_select.value, name='n1_nvme')
        node1_gpu = node1.add_component(ctype=ComponentType.GPU, model=gpu_select.value, name='n1_nvme')
        node1_nic = node1.add_component(ctype=ComponentType.SharedNIC, model=nic_slect.value, name='n1_nvme')
    except:
        pass
    
    if node1_nvme:
        viewer.append_stdout('nvme submitted')
    if node1_gpu:
        viewer.append_stdout('gpu submitted')
    if node1_nic:
        viewer.append_stdout('nic submitted')
    


submit_component_button = widgets.Button(
    descriptions='Submit Component',
    disabled=False,
    tooltip='Click to submit component'
)
    
def add_components(self):
    with viewer:
        display(nvme_name)
        display(nvme_select)
        display(gpu_name)
        display(gpu_select)
        display(nic_name)
        display(nic_select)
        display(submit_component_button)
        
add_components_button = widgets.Button(
    description='Add Components',
    disabled=False,
    tooltip='Click to Add Components',
)
add_components_button.on_click(add_components)

def add_node(self):
    with viewer:        
        left_box = widgets.VBox([node_name, 
                                 site_select, 
                                 core_slider,
                                 ram_slider,
                                 disk_slider
                                ])
        right_box = widgets.VBox([img_ref_select,
                                  img_type_select
                                 ])
        display(widgets.HBox([left_box, right_box]))
        display(widgets.HBox([add_components_button, preview_request]))


add_node_button = widgets.Button(
    description='Add Node',
    disabled=False,
    tooltip='Click to Add Node'
)
add_node_button.on_click(add_node)

def build_slice_request(self):
    with viewer:
        display(add_node_button)
    
build_req_button = widgets.Button(
    description='Build Slice Request',
    disabled=False,
    tooltip='Click to Build Slice Request',
)
build_req_button.on_click(build_slice_request)


NameError: name 'widgets' is not defined

In [ ]:

with viewer:
    viewer.append_stdout('PROJECT VIEWER')
    display(widgets.HBox([ad_res_button, rec_top_button, build_req_button, clear_button]))
            